In [34]:
import os
import polars as pl
from pathlib import Path
import pandas as pd
import numpy as np
from lxml import etree # type: ignore <- pylance milně hlásí chybu
import random


os.chdir(r'E:\CVUT_BAP')

In [ ]:
downloaded = os.listdir('kod/data/data_z_mericich_pristroju/gz')
len(downloaded)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'kod/data/data_z_mericich_pristroju/gz'

In [26]:
parsed = os.listdir('kod/data/data_z_mericich_pristroju/xml')
len(parsed)

2513

In [27]:
parsed = os.listdir('kod/data/prohlidky_vozidel_stk_a_sme/parquet/prohlidky')
len(parsed)

2517

In [12]:
for file in downloaded:
    path = os.path.join('stazeni/datove_sady_stk/', file)
    os.remove(path)

In [ ]:
# for file in extracted:
#     path = os.path.join('stazeni/vybrane_datove_sady_stk/', file)
#     os.remove(path)

In [15]:
# 1. Define the unified schema for the conflicting column(s)
prohlidky_schema = {
    # --- Primary Inspection Fields ---
    "CisloProtokolu": pl.String,
    "DatumProhlidky": pl.String, # Assuming date is stored as 'YYYY-MM-DD' string initially
    "DruhProhlidky": pl.String,
    "RozsahProhlidky": pl.String, # Fixes the SchemaError (enforcing String type)
    "Prohlidka_OdpovednaOsoba": pl.String, # Employee ID (stored as string)
    
    # --- Stanice (Station) Fields ---
    "Prohlidka_Stanice_Cislo": pl.String,
    "Prohlidka_Stanice_Kraj": pl.String,
    "Prohlidka_Stanice_ORP": pl.String,
    "Prohlidka_Stanice_Obec": pl.String,

    # --- Casove Udaje (Timestamp Fields) ---
    "Prohlidka_Zahajeni": pl.String,
    "Prohlidka_Ukonceni": pl.String,

    # --- Administrativni Oprava Fields ---
    "AdministrativniOprava_CisloProtokolu": pl.String,
    "AdministrativniOprava_DatumProhlidky": pl.String,

    # --- Vozidlo (Vehicle) Fields ---
    "Vozidlo_Vin": pl.String,
    "Vozidlo_Druh": pl.String,
    "Vozidlo_Kategorie": pl.String,
    "Vozidlo_Provedeni": pl.String,
    "Vozidlo_Znacka": pl.String,
    "Vozidlo_ObchodniOznaceni": pl.String,
    "Vozidlo_TypMotoru": pl.String,

    # --- Registrace (Registration) Fields ---
    "Registrace_DatumPrvni": pl.String,
    "Registrace_Stat": pl.String,
    "Registrace_CisloDokladu": pl.String,

    # --- Emisni Cast (Emissions) Fields ---
    "Emise_CisloProtokolu": pl.String,
    "Emise_DatumProhlidky": pl.String,
    "Emise_Stanice_Cislo": pl.String,
    "Emise_Stanice_Kraj": pl.String,
    "Emise_Stanice_ORP": pl.String,
    "Emise_Stanice_Obec": pl.String,
    "Emise_Zahajeni": pl.String,
    "Emise_Ukonceni": pl.String,
    "Emise_OdpovednaOsoba": pl.String,
    "Emise_ZakladniPalivo": pl.String,
    "Emise_AlternativniPalivo": pl.String,
    "Emise_EmisniSystem": pl.String,
    "Emise_VyrobceMotoru": pl.String,
    "Emise_CisloMotoru": pl.String,
    "Emise_RokVyroby": pl.String,

    # --- Technicka Cast (Technical) Fields ---
    "Technicka_Zahajeni": pl.String,
    "Technicka_Ukonceni": pl.String,
    "Technicka_OdpovednaOsoba": pl.String,

    # --- Adr Cast (ADR) Fields ---
    "Adr_Zahajeni": pl.String,
    "Adr_Ukonceni": pl.String,
    "Adr_OdpovednaOsoba": pl.String,
    "Adr_Platnost_Periodicka": pl.String,
    "Adr_Platnost_Meziperiodicka": pl.String,
    "Adr_KodCisterny": pl.String,
    "Adr_CisloOsvedceni": pl.String,
    "Adr_ZavadyText": pl.String,
    "Adr_Poznamka": pl.String,

    # --- Tsk Cast (Specific Inspection) Fields ---
    "Tsk_Zahajeni": pl.String,
    "Tsk_Ukonceni": pl.String,
    "Tsk_OdpovednaOsoba": pl.String,

    # --- Vysledek (Result) Fields ---
    "Vysledek_Odometr": pl.String, # Odometer readings often stored as strings/integers
    "Vysledek_Poznamka": pl.String,
    "Vysledek_DatumPristiProhlidky": pl.String, # Date stored as string
    "Vysledek_NalepkaVylepena": pl.String,
    "Vysledek_Celkovy": pl.String,
}

# 2. Start a lazy query plan, enforcing the schema
lazy_df = pl.scan_parquet(
    r'kod\data\prohlidky_vozidel_stk_a_sme\parquet\prohlidky',
    schema=prohlidky_schema  # Enforce the defined schema
)

# 3. Apply filters/transformations (Your query logic)
result_query = (
    lazy_df
    .filter(pl.col("Vozidlo_Vin") == "WVWZZZ7NZDV000614")
    .sort('Technicka_Zahajeni')
    .limit(1000)
)

# 4. Execute the query
final_result = result_query.collect()
display(final_result[-10:])

CisloProtokolu,DatumProhlidky,DruhProhlidky,RozsahProhlidky,Prohlidka_OdpovednaOsoba,Prohlidka_Stanice_Cislo,Prohlidka_Stanice_Kraj,Prohlidka_Stanice_ORP,Prohlidka_Stanice_Obec,Prohlidka_Zahajeni,Prohlidka_Ukonceni,AdministrativniOprava_CisloProtokolu,AdministrativniOprava_DatumProhlidky,Vozidlo_Vin,Vozidlo_Druh,Vozidlo_Kategorie,Vozidlo_Provedeni,Vozidlo_Znacka,Vozidlo_ObchodniOznaceni,Vozidlo_TypMotoru,Registrace_DatumPrvni,Registrace_Stat,Registrace_CisloDokladu,Emise_CisloProtokolu,Emise_DatumProhlidky,Emise_Stanice_Cislo,Emise_Stanice_Kraj,Emise_Stanice_ORP,Emise_Stanice_Obec,Emise_Zahajeni,Emise_Ukonceni,Emise_OdpovednaOsoba,Emise_ZakladniPalivo,Emise_AlternativniPalivo,Emise_EmisniSystem,Emise_VyrobceMotoru,Emise_CisloMotoru,Emise_RokVyroby,Technicka_Zahajeni,Technicka_Ukonceni,Technicka_OdpovednaOsoba,Adr_Zahajeni,Adr_Ukonceni,Adr_OdpovednaOsoba,Adr_Platnost_Periodicka,Adr_Platnost_Meziperiodicka,Adr_KodCisterny,Adr_CisloOsvedceni,Adr_ZavadyText,Adr_Poznamka,Tsk_Zahajeni,Tsk_Ukonceni,Tsk_OdpovednaOsoba,Vysledek_Odometr,Vysledek_Poznamka,Vysledek_DatumPristiProhlidky,Vysledek_NalepkaVylepena,Vysledek_Celkovy
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""CZ-550507-22-07-0106""","""2022-07-08""","""Pravidelná""",null,"""3429""","""550507""","""Liberecký kraj""","""Liberec""","""Český Dub""","""2022-07-08T07:59:19.1600000+02…","""2022-07-08T10:09:00.2500000+02…",null,null,"""WVWZZZ7NZDV000614""","""OSOBNÍ AUTOMOBIL""","""M1""","""AC KOMBI""","""VOLKSWAGEN""","""SHARAN""","""CFF""","""2012-08-02T00:00:00.0000000+02…","""Česká republika""","""UE917213""","""CZ-550507-22-07-0106""","""2022-07-08T10:09:00.2500000+02…","""550507""",null,null,null,"""2022-07-08T09:52:50.5930000+02…","""2022-07-08T10:08:29.2330000+02…","""3429""","""Nafta""",null,"""Řízený s OBD""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""169150""",null,"""2024-07-08""",null,"""1"""
"""CZ-410945-20-06-0351""","""2020-06-09""","""Pravidelná""",null,"""36701""","""410945""","""Hlavní město Praha""","""Hlavní město Praha""","""Praha""","""2020-06-09T12:28:40.7570000+02…","""2020-06-09T12:36:31.9230000+02…",null,null,"""WVWZZZ7NZDV000614""","""OSOBNÍ AUTOMOBIL""","""M1""",null,"""VOLKSWAGEN""","""SHARAN""","""CFF""","""2012-08-02T00:00:00.0000000+02…","""Česká republika""","""UE917213""","""CZ-410945-20-06-0351""","""2020-06-09T12:36:31.9230000+02…","""410945""",null,null,null,"""2020-06-09T12:31:49.1430000+02…","""2020-06-09T12:36:25.5770000+02…","""36701""","""Nafta""",null,"""Řízený s OBD""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""145479""",null,"""2022-06-09""",null,"""1"""
"""CZ-550507-25-02-0391""","""2025-02-22""","""Pravidelná""",null,"""35171""","""550507""","""Liberecký kraj""","""Liberec""","""Český Dub""","""2025-02-22T10:18:49.3630000+01…","""2025-02-22T10:41:35.1930000+01…",null,null,"""WVWZZZ7NZDV000614""","""OSOBNÍ AUTOMOBIL""","""M1""",null,"""VOLKSWAGEN""","""SHARAN""","""CFF""","""2012-08-02T00:00:00.0000000+02…","""Česká republika""","""UAU352178""","""CZ-550507-25-02-0391""","""2025-02-22T10:41:35.1930000+01…","""550507""",null,null,null,"""2025-02-22T10:30:44.8430000+01…","""2025-02-22T10:40:44.6930000+01…","""35171""","""Nafta""",null,"""Řízený s OBD""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""210794""",null,"""2027-02-22""",null,"""1"""
"""CZ-3120-20-06-0753""","""2020-06-09""","""Pravidelná""","""Plný""","""17325""","""3120""","""Hlavní město Praha""","""Hlavní město Praha""","""Praha""","""2020-06-09T12:43:27.0800000+02…","""2020-06-09T13:32:29.9670000+02…",null,null,"""WVWZZZ7NZDV000614""","""OSOBNÍ AUTOMOBIL""","""M1""",null,"""VOLKSWAGEN""","""SHARAN""","""CFF""","""2012-08-02T00:00:00.0000000+02…",

In [12]:
import glob

files = glob.glob(r'kod\data\prohlidky_vozidel_stk_a_sme\parquet\prohlidky\*', recursive=True)
bad_files = []

for f in files:
    try:
        pl.read_parquet(f, n_rows=1)
    except Exception as e:
        bad_files.append((f, str(e)))

bad_files

[('kod\\data\\prohlidky_vozidel_stk_a_sme\\parquet\\prohlidky\\Prohlídky vozidel STK a SME za 01-06-2019.parquet',
  "parquet: File out of specification: The file must end with PAR1\n\nThis error occurred with the following context stack:\n\t[1] 'parquet scan'\n\t[2] 'sink'\n")]

In [11]:
files

['kod\\data\\prohlidky_vozidel_stk_a_sme\\parquet\\prohlidky']

In [5]:
mereni_2020_path = Path('../zdroje/ANALYZA DAT MERENI SME z ISTP/2020/sme')
csv_files = []
for f in mereni_2020_path.iterdir():
    if f.stem[-1: ] == 'V':
        csv_files.append(f)


data_frames = []
for csv_file in csv_files:
    data_frames.append(pd.read_csv(csv_file, delimiter=';'))

df = pd.concat(data_frames)


C:\Users\adamp\AppData\Local\Temp\ipykernel_23496\3016768282.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(data_frames)


In [28]:
# print(list(df.columns))

In [10]:
df[np.all(df[['D11-AccTime', 'D12-AccTime', 'D13-AccTime', 'D14-AccTime']].values == [1.88, 1.84, 1.78, 1.70], axis=1)][['SME', 'DateTime', 'D11-AccTime','D12-AccTime', 'D13-AccTime', 'D14-AccTime', 'VIN']]

,SME,DateTime,D11-AccTime,D12-AccTime,D13-AccTime,D14-AccTime,VIN
73,540514.0,2020-01-03T06:40:00,1.88,1.84,1.78,1.7,VF3GJRHYK95052878
182,540514.0,2020-01-06T10:10:00,1.88,1.84,1.78,1.7,ZFA19200000105264
204,540514.0,2020-01-06T10:20:00,1.88,1.84,1.78,1.7,WBAAL71020KE07146
207,540514.0,2020-01-06T10:31:00,1.88,1.84,1.78,1.7,YV1SW720211084029
221,540514.0,2020-01-06T10:31:00,1.88,1.84,1.78,1.7,YV1SW720211084029
...,...,...,...,...,...,...,...
18378,540514.0,2020-08-28T09:51:00,1.88,1.84,1.78,1.7,WDB6013671P076453
18427,540514.0,2020-08-31T06:20:00,1.88,1.84,1.78,1.7,WVWZZZ1HZXK024417
18429,540514.0,2020-08-31T06:11:00,1.88,1.84,1.78,1.7,TMBBG41U432699607
18454,540514.0,2020-08-31T08:17:00,1.88,1.84,1.78,1.7,WVWZZZ3BZ1E253013


In [9]:

vin = "WVWZZZ3BZ1E253013"

# lazy scan, filter only VIN matches
df = (
    pl.scan_csv(
        "../zdroje/RSV_vypis_vozidel_20251101.csv",
        infer_schema_length=1000,
        truncate_ragged_lines=True,
        ignore_errors=True,
        quote_char=None
    )
    .filter(pl.any_horizontal(pl.all().cast(pl.Utf8).str.contains(vin)))
    .collect()
)

df.shape[0] > 0  # True if VIN exists

True

In [10]:
df

Datum 1 registrace,Datum 1 registrace v ČR,ZTP,ES EU,Druh vozidla,Druh vozidla 2 ř,Kategorie vozidla,Tovární značka,Typ,Varianta,Verze,VIN,Obchodní označení,Výrobce vozidla,Výrobce motoru,Typ motoru,Max výkon kW min⁻¹,Palivo,Zdvihový objem cm³,Plně elektrické vozidlo,Hybridní vozidlo,Třída hybridního vozidla,Emisní limit EHKOSN EHSES,Stupeň plnění emisní úrovně,Korigovaný součinitel absorbce m⁻¹,CO2 město mimo město kombinované g km-1,Specifické CO2,Snížení emisí – NEDC,Snížení emisí – WLTP,Spotřeba předpis,Spotřeba město mimo město kombinovaná l 100km⁻¹,Spotřeba při rychlosti l 100 km⁻¹,Spotřeba el mobil Wh km – Z,Dojezd ZR km,Výrobce karoserie,Druh (typ),Výrobní číslo karoserie,…,Faktor verifikace - Vf,Účel vozidla,Další záznamy,Alternativní provedení,Číslo TP,Číslo ORV,Druh RZ,Zařazení vozidla,Status,PČV,ABS,Airbag,ASR,Brzdy nouzová,Brzdy odlehčovací,Brzdy parkovací,Brzdy provozní,Doplňkový text na TP,Hmotnosti provozní Do,Hmotnosti zatížení SZ,Hmotnosti zatížení SZ typ,Hydropohon,Objem cisterny,Zatížení střechy,Číslo motoru,Nejvyšší rychlost omezení,Ovládání brzd SZ,Ovládání brzd SZ druh,Retarder,Rok výroby,Délka do,Ložná délka,Ložná šířka,Výška do,Typ kód,RM zániku,Stupeň autonomity vozidla
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""11.04.2001""","""25.10.2012""","""3483-137-00""","""e1*98/14*0157*""","""OSOBNÍ AUTOMOBIL""","""OSOBNÍ KOMBI""","""M1""","""VW""","""""/ 3BG""""",null,null,"""WVWZZZ3BZ1E253013""","""PASSAT VARIANT""","""""""""","""""VOLKSWAGEN""",""" AG""",""" WOLFSBURG""",""" SRN""""","""AVF""","""""96 / 4000""""","""NM""","""""1896""""","""NE""","""NE""",null,""" / 1999/102A""",null,"""""0""","""9""""",""" / / """,null,null,null,"""""EU 1999/100""""","""""8.6 / 5.5 / 6.6""""","""""0 /""""",null,…,"""""195/65 R 15 91 V / 6J X 15 H2…","""""78/ 3000""""","""74""","""201""",null,null,null,null,null,"""Běžný provoz""","""""Variabilní provedení vozidla:…",""" ve znění pozdějších předpisů.…","""| - lapači nečistot""","""| - na výrobním štítku označen…","""|hmotností nebrzděného přívěsu…",null,"""UE572454""","""UAS198182""","""STD. SILNIČNÍ""","""RSV""","""PROVOZOVANÉ""","""12347465""","""True""",null,"""False""","""False""","""False""","""True""","""True""","""""DOVOZ SRN Č.POJ.:CZ/0008/8797…",null,"""75""",null,"""False""",null,"""100""",null


In [91]:
def safe_get(element, xpath, namespaces):
    if element is None:
        return None
    node = element.find(xpath, namespaces)
    return node.text if node is not None else None

def safe_find(element, xpath, namespaces):
    if element is None:
        return None
    return element.find(xpath, namespaces)

def safe_findall(element, xpath, namespaces):
    if element is None:
        return None
    return element.findall(xpath, namespaces)

namespaces = {
    'm': 'istp:opendata:schemas:MereniSeznam:v1', 
    'd': 'istp:opendata:schemas:DatovaSada:v1'      
}

merici_pristroje = Path('kod/data/data_z_mericich_pristroju/xml')
files = list(merici_pristroje.glob('*.xml'))
random.shuffle(files)
result = set()
for i, file in enumerate(files):
    if i >= 100:
        break
    print('.', end='')
    tree = etree.parse(file)
    mereni_seznam = safe_findall(safe_find(tree.find('d:DatovyObsah', namespaces), 'm:MereniSeznam', namespaces), 'm:Mereni', namespaces)
    if mereni_seznam is None:
        continue
    for mereni in mereni_seznam:
        parent = safe_findall(safe_find(safe_find(mereni, 'm:PristrojData', namespaces), 'm:detailNafta', namespaces), 'm:vyusteni', namespaces)#, 'm:otackyZvysene', namespaces)#, 'm:J1939', namespaces)#, 'm:HC', namespaces)
        if parent is None:
            continue

        # result = result | set(str(len(list(parent))))
      
        result = result | set([len(parent)])
        continue
        children = list(parent)
        names = {child.tag.split('}')[-1] for child in children}
        # if len(names) > 0:
        #     print(file)
        #     print(safe_get(mereni, 'm:CisloProtokolu', namespaces))
        result = result | names


    
display(result)



....................................................................................................

{0, 1, 2, 3}

In [41]:
display(result)

{'mereniVznetove', 'mereniZazehove', 'nevyhovuje', 'vyhovuje'}

In [46]:
display(result)

{'mereniVznetLimit', 'mereniVznetVysledek'}